# 词向量与词嵌入

本章介绍词嵌入（word embedding）方法。依次有如下内容：
1. 概述
2. 机器学习对于词的表示方法
3. 神经网络语言模型（Neural Network Language Model）
4. word2vec构造词向量
5. 使用keras完成词向量的训练与可视化

### 机器学习对于词的表示方法

在NLP建模中，最重要和最根本的任务就是在模型中对输入的单词进行表示，方便模型识别单词之间的相似性和差异性。词的表达主要有两大类：早期的NLP工作中，词的表示大多是基于基本符号（atomic symbol）；而在现代NLP中，更多的是以词向量（word vector）的方法来表示，从而使得词间相似性计算更加便捷。下面先简要介绍早期的词表示方法，再着重解释现代的词向量方法。

#### 词向量方法

在真实世界中，单词的数量是巨大的。比如根据现代汉语词典，中文里常用的基本词和词组大约有5万6千多个，加上现在网络上新兴的各种单词及其组合，则无人有过精确统计；至于英语，韦伯斯特词典则大约收录了1百万个英语单词。当然，汉语的词数量不适宜跟英语相比，中文的词主要是一个词根作用，加上其各种组合应该数量也是巨大的。

这些单词之间并不是完全独立的，也不是每一个单词都代表一个完全独立的概念。我们可以认为我们在谈话时所表达的实际语义实际更少，是在一个数量更低的维度上。比如性别（男 vs 女）；交通工具（汽车 vs 飞机），食品（宫保鸡丁 vs 火锅）等等。

我们下面介绍几种常见的词向量表示方法：
1. 独热编码（One Hot Encoding）
2. 基于SVD的编码
3. 迭代编码方法

#### 独热编码 （One Hot Encoding）

独热编码是对于当前单词表中的单词使用一个向量进行表达的简单方式。假如当前的单词表有10个不同的单词，那么独热编码使用10位的0、1数字来对该单词表进行编码，出现每个单词在对应的序号位标为1，否则为0。下面举例说明：

假设我们的文档分词后产生如下的单词表：[“中国”，“国家”，“主席”，“习近平”，“北京”，“钓鱼台”，“会见”，“到”，“访”，“日本”，“首相”]，共11个单词，并且单词序号也按照上面的次序，那么我们的独热编码$w$就使用一个11个0、1数字的向量来表示这些单词：

w(“中国”)：1000000000； w(“国家”)：01000000000，......，w(“首相”)：00000000001

一般说来，对于一个具有N个元素的单词表，独热编码将每一个单词映射为$R^{1\times|N|}$的向量，向量中对应单词序号的位置数值为1，其余为0。

当然，独热编码虽然简单，但是其有几个问题：
1. 存储效率极低
2. 每个单词是完全独立的存在，之间并无联系。比如$w （“中国”）^T w（“日本”）=0$



#### 基于SVD的编码
我们看到，独热编码虽然简单，但是对于建模来说并不是较好的选择。研究者探索了可以解决存储效率和单词之间关系问题的方法，其中基于SVD的编码便是较常见的一种。这种方法依赖于通过word-document矩阵生成的共生矩阵（cooccurance matrix），下面分别介绍：
1. 词-文矩阵（word-document matrix）。这是以单词为行，以文本为列的矩阵，反映一个文本中以0、1编码表示的所有单词；
2. 基于滑动窗的共生矩阵（window based co-occurance matrix）。该矩阵是一个“词-词矩阵”（word-word matrix），反映的是在一个给定的窗口中两个单词同时出现的频次统计；

直接按照逻辑来实现对以上矩阵的生产并不难，但是sklearn里面已经有现成的方法可供调用，直接对以列表形式出现的文本集合进行操作，非常方便。下面先举一个简单示例说明逻辑，再展示实践中可用的生产性代码。在英文文档中，可以直接使用sklearn的文字处理模块进行操作，但是中文需要先进行分词再使用sklearn的模块进行处理。

下面，我们：
- 首先展示如何生成**词-文矩阵**；
- 再展示如何生成基于滑动窗的**共生矩阵**；


In [1]:
# 对于英文文档列表，可以直接使用sklearn工具生成 WORD-DOCUMENT Matrix 
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from IPython.display import SVG, display

%matplotlib inline

docs = ['this is a good book',
        'this cat is this good',
        'cat dog fight']
count_model = CountVectorizer(ngram_range=(1,1)) # default unigram model
X = count_model.fit_transform(docs)
print(count_model.get_feature_names())
print(X.todense())
print(count_model.vocabulary_)

['book', 'cat', 'dog', 'fight', 'good', 'is', 'this']
[[1 0 0 0 1 1 1]
 [0 1 0 0 1 1 2]
 [0 1 1 1 0 0 0]]
{'this': 6, 'is': 5, 'good': 4, 'book': 0, 'cat': 1, 'dog': 2, 'fight': 3}


**对中文进行“词-文矩阵”生成**

英文没有分词的要求，所有单词由空格自然分开；但是中文由于其特殊性，需要进行额外分词。这里我们使用流行的“结巴分词”（jieba）软件包来进行分词，读者也可以选择自己的分词软件。

从上面的英文例子我们看到，对于中文文本，只需要将连续的中文使用jieba进行分词，将单词用空格隔开后放入一个字符串即可使用sklearn的现成方法生成词-文矩阵。下面我们进行演示。第一步我们需要将原有中文字符串列表中的中文字符串进行分词，再用空格连接后形成新的字符串放回列表中。

根据jieba分词库的自我介绍，其支持三种分词模式：
- 精确模式，试图将句子最精确地切开，适合文本分析，也是最常用的。该模式对应jieba.cut方法，使用**cut_all=False**选项
- 全模式，把句子中所有的可以成词的词语都扫描出来, 速度非常快而且颗粒度非常高，但是不能解决歧义，比如“...川普通过...”会被切分为“川普”，“普通”等多个词，但是这里实际上只有“川普”这个词有意义。该模式也对应jieba.cut方法，使用**cut_all=True**选项
- 搜索引擎模式，在精确模式的基础上，对长词再次切分，提高召回率，适合用于搜索引擎分词。该模式对应**jieba.cut_for_search**方法。

在jieba3k中，以上两种方法都返回一个可迭代的生成器（generator）对象，可以使用for 循环来获取分割后的单词。jieba分词会将阿拉伯数字单独分割出来，如果需要将这些阿拉伯数字挑出来，可以使用bool(re.search('\d+', word))来判断。

在老版本的jieba.cut和jieba.cut_for_search各自对应一个返回列表的函数，分别是lcut 和lcut_for_search，其用法一样，只是返回数据类型是一个列表，但是在jieba3k中似乎没有这些方法了。

jieba也自带一个Tokenizer对象，可新建一个自定义的分词器，使用不同的词典，这里不再详述。

对于我们的应用来说，只需要选好一个分词模式，对于原有中文字符串列表进行迭代，对于每一个字符串分词后的单词用空格连接起来再放回列表即可，下面的代码演示了采用精确模式进行分词的应用。

In [2]:
import jieba
import re
jieba.initialize()  
# 简单示例
documents = [u'独热编码是对于当前单词表中的单词使用1个向量进行表达的简单方式，独热编码有自身的缺点和有点。', 
             u'假如当前的单词表有10个不同的单词，并且每个单词都不一样', 
             u'研究者探索了可以解决存储效率和单词之间关系问题的方法', 
             u'独热编码虽然简单']

documents_after = []

# 第一步先将中文词切分开，每个原有中文字符串组成一个单独的列表
documents_after = [[w for w in jieba.cut(s)] for s in documents]
# 第二部用空格将单独列表中的中文词元素连接成一个带空格的字符串，从而可以模仿英文的操作
documents_after = [ ' '.join(s) for s in documents_after]
print(documents_after)
print(len(documents_after))

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\xieliang\AppData\Local\Temp\jieba.cache
Loading model cost 0.881 seconds.
Prefix dict has been built succesfully.


['独热 编码 是 对于 当前 单词表 中 的 单词 使用 1 个 向量 进行 表达 的 简单 方式 ， 独热 编码 有 自身 的 缺点 和 有点 。', '假如 当前 的 单词表 有 10 个 不同 的 单词 ， 并且 每个 单词 都 不 一样', '研究者 探索 了 可以 解决 存储 效率 和 单词 之间 关系 问题 的 方法', '独热 编码 虽然 简单']
4


下面进行矩阵构建操作，这里我们使用类似处理英文的标准方法。

In [3]:
print(u'中文分词后的文档：')
print(documents_after)
print('\n')

# 再使用sklearn模块生成word-document矩阵
cn_count_model = CountVectorizer(ngram_range=(1,1), lowercase=False) # default unigram model, 缺省要小写变换在中文中不适用，会报错
cnX = cn_count_model.fit_transform(documents_after)

print(u'标注化（Tokenized）后的特征列表：')
print(cn_count_model.get_feature_names())
print('\n')

print(u'词-文矩阵：')
print(cnX.todense())
print('\n')

print(u'词-文矩阵对应的特征索引号（矩阵列的序号）：')
print(cn_count_model.vocabulary_)
print('\n')

# 将词-文矩阵的列打上标签：
voc_df=pd.DataFrame.from_dict(cn_count_model.vocabulary_, columns=['idx'], orient='index').sort_values(by=['idx'])
cols = list(voc_df.index)
print('带标签的词-文矩阵:')
pd.DataFrame(cnX.todense(), columns=cols)

中文分词后的文档：
['独热 编码 是 对于 当前 单词表 中 的 单词 使用 1 个 向量 进行 表达 的 简单 方式 ， 独热 编码 有 自身 的 缺点 和 有点 。', '假如 当前 的 单词表 有 10 个 不同 的 单词 ， 并且 每个 单词 都 不 一样', '研究者 探索 了 可以 解决 存储 效率 和 单词 之间 关系 问题 的 方法', '独热 编码 虽然 简单']


标注化（Tokenized）后的特征列表：
['10', '一样', '不同', '之间', '使用', '假如', '关系', '单词', '单词表', '可以', '向量', '存储', '对于', '并且', '当前', '探索', '效率', '方式', '方法', '有点', '每个', '独热', '研究者', '简单', '编码', '缺点', '自身', '虽然', '表达', '解决', '进行', '问题']


词-文矩阵：
[[0 0 0 0 1 0 0 1 1 0 1 0 1 0 1 0 0 1 0 1 0 2 0 1 2 1 1 0 1 0 1 0]
 [1 1 1 0 0 1 0 2 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 1 1 0 1 0 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 0]]


词-文矩阵对应的特征索引号（矩阵列的序号）：
{'独热': 21, '编码': 24, '对于': 12, '当前': 14, '单词表': 8, '单词': 7, '使用': 4, '向量': 10, '进行': 30, '表达': 28, '简单': 23, '方式': 17, '自身': 26, '缺点': 25, '有点': 19, '假如': 5, '10': 0, '不同': 2, '并且': 13, '每个': 20, '一样': 1, '研究者': 22, '探索': 15, '可以': 9, '解决': 29, '存储': 11, '效率': 16, '之间': 3, '关系': 6, '

,10,一样,不同,之间,使用,假如,关系,单词,单词表,可以,...,研究者,简单,编码,缺点,自身,虽然,表达,解决,进行,问题
0,0,0,0,0,1,0,0,1,1,0,...,0,1,2,1,1,0,1,0,1,0
1,1,1,1,0,0,1,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,1,1,0,1,...,1,0,0,0,0,0,0,1,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,1,0,0,0,0


我们发现上述操作是可行的，因此可以将第一步的操作封装为一个函数供后续步骤使用。

In [4]:
documents2 = [u'独热编码是对于当前单词表中的单词使用1个向量进行表达的简单方式，独热编码有自身的缺点和有点。']
documents2_after = [[w for w in jieba.cut(s)] for s in documents2]
documents2_after = [ ' '.join(s) for s in documents2_after]
print(documents2_after)

['独热 编码 是 对于 当前 单词表 中 的 单词 使用 1 个 向量 进行 表达 的 简单 方式 ， 独热 编码 有 自身 的 缺点 和 有点 。']


In [5]:
def cn_string_to_words(documents):
    # 第一步先将中文词切分开，每个原有中文字符串组成一个单独的列表
    documents_after = [[w for w in jieba.cut(s)] for s in documents]
    # 第二部用空格将单独列表中的中文词元素连接成一个带空格的字符串，从而可以模仿英文的操作
    documents_after = [ ' '.join(s) for s in documents_after]
    return documents_after

但是在实际工作中存在大量文本，我们会发现不同词的出现频率非常不同。这时候单纯使用词-文矩阵里面的频率计数到模型中会造成有偏差的结果。事实上，如果一个不常见的词在某一个文本中出现，那么其携带的信息量反而是非常高的。一种常见的变换方法就是TF-IDF（term-frequency inverse document frequency）。这个方法将高频词按照在所有文本中出现的频次进行降低权重的操作，从而来“突出”低频词的作用。tfidf使得文本相对更可比，计算文本之间的相似性的时候更有意义。

TFIDF在sklearn里面有现成的工具进行操作。

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

cn_tf_model = TfidfVectorizer()
X = cn_tf_model.fit_transform(documents_after)
print(cn_tf_model.get_feature_names())
print(X.shape)

voc_df=pd.DataFrame.from_dict(cn_tf_model.vocabulary_, columns=['idx'], orient='index').sort_values(by=['idx'])
cols = list(voc_df.index)
print('带标签的词-文矩阵:')
Xdf=pd.DataFrame(X.todense(), columns=cols)
# 现在行对应单词，列对应文本
Xdf.T


['10', '一样', '不同', '之间', '使用', '假如', '关系', '单词', '单词表', '可以', '向量', '存储', '对于', '并且', '当前', '探索', '效率', '方式', '方法', '有点', '每个', '独热', '研究者', '简单', '编码', '缺点', '自身', '虽然', '表达', '解决', '进行', '问题']
(4, 32)
带标签的词-文矩阵:


,0,1,2,3
10,0.000000,0.335714,0.000000,0.000000
一样,0.000000,0.335714,0.000000,0.000000
不同,0.000000,0.335714,0.000000,0.000000
之间,0.000000,0.000000,0.309976,0.000000
使用,0.248108,0.000000,0.000000,0.000000
假如,0.000000,0.335714,0.000000,0.000000
关系,0.000000,0.000000,0.309976,0.000000
单词,0.158364,0.428563,0.197854,0.000000
单词表,0.195611,0.264680,0.000000,0.000000
可以,0.000000,0.000000,0.309976,0.000000


#### 滑动窗口的共生矩阵生成
滑动窗口的共生矩阵因为包含了两个单词在一定相邻距离上同时出现的频次，因此反映了单词之间的相关程度。

如果不考虑窗口大小，从而在全局考虑共生矩阵的生成的化，只需要使用词-文矩阵的矩阵乘积即可，下面展示：

In [7]:
documents = [u'独热编码是对于当前单词表中的单词使用1个向量进行表达的简单方式，独热编码有自身的缺点和有点。', 
             u'假如当前的单词表有10个不同的单词，并且每个单词都不一样', 
             u'研究者探索了可以解决存储效率和单词之间关系问题的方法', 
             u'独热编码虽然简单']
documents_after  = cn_string_to_words(documents)

# 再使用sklearn模块生成word-document矩阵
min_n = 1
max_n = 1
cn_count_model = CountVectorizer(ngram_range=(min_n, max_n), lowercase=False) # 缺省为 unigram 模型
cnX = cn_count_model.fit_transform(documents_after)


print(u'共生矩阵：')
#cnXdense = cnX.todense()
#%time XTX = np.dot(cnXdense.T, cnXdense)
%time XTX = cnX.T * cnX
print(XTX.todense(), '\n')


print(u'词-文矩阵对应的特征索引号（矩阵列的序号）：')
print(cn_count_model.vocabulary_)
print('\n')

# 将共生矩阵的列打上标签：
voc_df=pd.DataFrame.from_dict(cn_count_model.vocabulary_, columns=['idx'], orient='index').sort_values(by=['idx'])
cols = list(voc_df.index)
print('带标签的共生矩阵:')
pd.DataFrame(XTX.todense(), columns=cols, index=cols)

共生矩阵：
Wall time: 0 ns
[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 1]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 1]] 

词-文矩阵对应的特征索引号（矩阵列的序号）：
{'独热': 21, '编码': 24, '对于': 12, '当前': 14, '单词表': 8, '单词': 7, '使用': 4, '向量': 10, '进行': 30, '表达': 28, '简单': 23, '方式': 17, '自身': 26, '缺点': 25, '有点': 19, '假如': 5, '10': 0, '不同': 2, '并且': 13, '每个': 20, '一样': 1, '研究者': 22, '探索': 15, '可以': 9, '解决': 29, '存储': 11, '效率': 16, '之间': 3, '关系': 6, '问题': 31, '方法': 18, '虽然': 27}


带标签的共生矩阵:


,10,一样,不同,之间,使用,假如,关系,单词,单词表,可以,...,研究者,简单,编码,缺点,自身,虽然,表达,解决,进行,问题
10,1,1,1,0,0,1,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
一样,1,1,1,0,0,1,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
不同,1,1,1,0,0,1,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
之间,0,0,0,1,0,0,1,1,0,1,...,1,0,0,0,0,0,0,1,0,1
使用,0,0,0,0,1,0,0,1,1,0,...,0,1,2,1,1,0,1,0,1,0
假如,1,1,1,0,0,1,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
关系,0,0,0,1,0,0,1,1,0,1,...,1,0,0,0,0,0,0,1,0,1
单词,2,2,2,1,1,2,1,6,3,1,...,1,1,2,1,1,0,1,1,1,1
单词表,1,1,1,0,1,1,0,3,2,0,...,0,1,2,1,1,0,1,0,1,0
可以,0,0,0,1,0,0,1,1,0,1,...,1,0,0,0,0,0,0,1,0,1


如果要生成含有特定窗口宽度的共生矩阵，这时我们需要自己写一个移动窗口函数，对于任意给定的单词（以下标index表示），在其左右给定窗口范围内获取邻近词。该操作可以使用如下的list comprehension操作实现：
start = index - window_size
end = index + window_size + 1
neighbor_words = [words[i] for i in range(start, end) if 0 <= i < L and i != index]

其中，L是字符串长度。注意，这里我们的输入是已经转换为下标的，以空格间隔的中文单词字符串。

下面，我们先使用一个双重list comprehension将中文单词依照单词表转换为下标。由于下标0在keras的所有文字处理方法中都是保留下标，程序不会认为该数字对应于一个单词，因此这里我们也将sklearn生成的下标值+1。

其次再编写一个函数，使用移动窗口技巧获取临近单词。

In [8]:
vocabulary = cn_count_model.vocabulary_
V = len(vocabulary)
documents_index = [[vocabulary[word]+1 for word in doc.split(' ') if word in vocabulary.keys()] for doc in documents_after]
print(documents_index)

[[22, 25, 13, 15, 9, 8, 5, 11, 31, 29, 24, 18, 22, 25, 27, 26, 20], [6, 15, 9, 1, 3, 8, 14, 21, 8, 2], [23, 16, 10, 30, 12, 17, 8, 4, 7, 32, 19], [22, 25, 28, 24]]


In [9]:
from keras.preprocessing import sequence
from keras.utils import np_utils
def generate_cooccurancemat(corpus, window_size, V):
    maxlen = window_size*2
    result = np.zeros([V, V], dtype=int)    
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            #获取index对应的近邻词列表
            contexts = []                 
            s = index - window_size
            e = index + window_size + 1            
            contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
            x = sequence.pad_sequences(contexts, maxlen=maxlen)
            #对于该共生组合，对于其相应矩阵位置计数+1
            for k in x[0]:
                if k!=0:
                    result[word-1, k-1] += 1                        
    return result

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
cooccurancemat = generate_cooccurancemat(documents_index, 3, V)
pd.DataFrame(cooccurancemat, columns=cols, index=cols)

,10,一样,不同,之间,使用,假如,关系,单词,单词表,可以,...,研究者,简单,编码,缺点,自身,虽然,表达,解决,进行,问题
10,0,0,1,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
一样,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
不同,1,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
之间,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
使用,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,1,0,1,0
假如,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
关系,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
单词,1,1,1,1,1,0,1,2,2,0,...,0,0,0,0,0,0,0,1,1,1
单词表,1,0,1,0,1,1,0,2,0,0,...,0,0,1,0,0,0,0,0,0,0
可以,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0


在上面的例子中因为信息量较小，两个词同时出现多次的情况没有，因此这里使用较大的外部数据来进行展示。我们使用的是1000条酒店评论数据。

In [11]:
def load_hotel_data():
    documents2 = []
    stopword = []
    datafile_pos = './nlp_data/hotel_reviews_data/1000_pos.txt'
    datafile_neg = './nlp_data/hotel_reviews_data/1000_neg.txt'
    stopwordfile = './nlp_data/hotel_reviews_data/stopWord.txt'

    # 先读入stopword
    #fo=open(stopwordfile, encoding='UTF-8')
    with open(stopwordfile, encoding='UTF-8') as fo:
        for line in fo:
           stopword.append(line.strip('\n'))

    print(stopword[70:87])

    # 再读入原始评论文档
    with open(datafile_pos, encoding='UTF-8') as fo:
        for line in fo:
           documents2.append(line.strip('\n'))

    with open(datafile_neg, encoding='UTF-8') as fo:
        for line in fo:
           documents2.append(line.strip('\n'))    
    return documents, stopword

documents2, stopwords = load_hotel_data();
print(documents2[:4])

['除非', '除了', '此', '此间', '此外', '从', '从而', '打', '待', '但', '但是', '当', '当着', '到', '得', '的', '的话']
['独热编码是对于当前单词表中的单词使用1个向量进行表达的简单方式，独热编码有自身的缺点和有点。', '假如当前的单词表有10个不同的单词，并且每个单词都不一样', '研究者探索了可以解决存储效率和单词之间关系问题的方法', '独热编码虽然简单']


如果滑动窗口大小设为1，则考虑全局的共生矩阵

In [12]:
# 首先分词
documents2_after = cn_string_to_words(documents2)

# 建立词表索引，用来将文字转换为下标，从而我们可以使用上面的generate_cooccurancemat函数
min_n = 1
max_n = 1
cn_count_model = CountVectorizer(ngram_range=(min_n, max_n), lowercase=False, stop_words = stopword)
cnX = cn_count_model.fit_transform(documents2_after)

vocabulary = cn_count_model.vocabulary_
# 考虑到阿拉伯数字没有多少意义，这里我们要先将阿拉伯数字从单词表中剔除。首先制作一个词典索引拷贝列表，
#其中只保留阿拉伯数字元素，再在原始词典中逐一pop出阿拉伯数字
#bool(re.search('\d+', word))
keys = list(vocabulary.keys())
vocabulary2 = vocabulary.copy()
numkeys = [k for k in keys if bool(re.search('\d+', k))]
for key in numkeys:
    vocabulary2.pop(key)

# 清洁工作
del(keys)
# 重新fit数据，但是numkeys会被添加到stopword里
stopword.extend(numkeys)
cn_count_model2 = CountVectorizer(ngram_range=(min_n, max_n), lowercase=False, stop_words = stopword)
cnX = cn_count_model2.fit_transform(documents2_after) 

vocabulary2 = cn_count_model2.vocabulary_
documents_index = [[vocabulary2[word]+1 for word in doc.split(' ') if word in vocabulary2.keys()] for doc in documents2_after]
print(documents_index[:3])

# 现在开始使用generate_cooccurancemat函数生成共生矩阵
print(u'使用generate_cooccurancemat函数生成共生矩阵：')
XTX = generate_cooccurancemat(documents_index, 10, len(vocabulary2.keys()))
print('\n')


# 将共生矩阵的列打上标签：
voc_df=pd.DataFrame.from_dict(vocabulary2, columns=['idx'], orient='index').sort_values(by=['idx'])
cols = list(voc_df.index)
print('带标签的词-文矩阵:')
pd.DataFrame(XTX, columns=cols, index=cols)

NameError: name 'stopword' is not defined

我们可以将共生矩阵进行SVD分解，获取每个单词在新空间上的映射向量

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=10, n_iter=10, random_state=42)
%time svd.fit(XTX)
u = svd.transform(XTX)

In [ ]:
plt.scatter(u[:, 0], u[:, 1], s=2)
plt.xlim(-2.25, 3.25)
plt.ylim(-2.25, 2.25)

** 这里结束传统的基于词频的表达方法，开始介绍基于预测的表达方法。**

### 迭代嵌入方法
我们在上面介绍了基于SVD的词嵌入方法。通过共生矩阵以及SVD算法，独热法表示的单词之间的关系得到抽象，并映射到较低维度的致密空间中。但是这种方法基于全局信息，对存储量需求大。我们现在介绍一种迭代学习的方法来将单词映射到一个新的包含了上下文关系的空间中。这种方法的典型代表叫word2vec。在word2vec这个方法中需要引入两个概念“中心词”（center word）和“上下文”（context）

以“资产富裕的人爱投资股票”这句话为例，其经过处理后得到如下的单词：\[ '资产','富裕'，'人', '爱', '投资', '股票' \]。如果选定*‘富裕’*这个单词，在模型中其被称为“中心词”，而其上下文就是前后的\[ '资产','人', '爱', '投资', '股票' \]。一个中心词的上下文被word2vec这个算法用来衡量其含义。如果我们在大量的文本中都发现类似于“资产”，“财富”，“投资”这样的词经常性地出现在*“富裕”*这个词的周边，就能推断*“富裕”*的含义。有相似上下文的词就在word2vec这个模型里具备相似的含义，可以被视作是同义词，其对应的词向量则应该距离上接近。

当然，在实践中，上下文通常被定义为中心词左右对称给定长度的窗口覆盖的词，如下图所示：

<img src="./pics/Chapter1-1.png" width="400">
"中心词与上下文的独热表示"

在word2vec中，词向量本身就是模型的参数，通过对数据的建模可以获得参数的值。word2vec包含两种不同的子模型：
1. 连续型模型（Continuous Bag Of Words Model, CBOW Model）
2. 跳跃型模型（Skip-Gram Model）

下面分别介绍。

** 连续型模型 （CBOW） **

在CBOW中，模型是根据上下文单词来预测中心词。其基本步骤如下：


其架构如下图所示（图源自于："Deep learning for sentence classification"）。图中标识分别为：
1. $x_{ik}$为独热表示的上下文单词
2. $W_{V \times N}$为输入层的权重矩阵，大小为$V \times N$，其中V是词典大小，而N是设定的词向量维度
3. $h_i$是由$x_{ik}$与权重矩阵$W$相乘的到的隐含层结果，大小为$1 \times N$的向量
4. $W^{'}_{N \times V}$为输出层的权重矩阵。注意，这里的$W^{'}$不是$W$矩阵的转置，而是另外一个全新的矩阵。
5. $y_j$为待预测的中心词的独热表示


<img src="./pics/Chapter1-CBOW.png" width="400">


CBOW的计算过程比较直接：

1. 首先，对于给定窗口长度m，上下文$x^{(c)}$用相应的独热法表示，对于待预测的中心词$y^{(c)}$也用相应的独热编码表示，那么我们的输入数据为：$(x^{(c-m)},...,x^{(c-1)}, x^{(c+1)},..., x^{(c+m)} )$
2. 其次，对于上述上下文单词，与输入层权重矩阵相乘，得到嵌入向量$v_{c-m} = Wx^{(c-m)}, v_{c-m+1} = Wx^{(c-m+1)}, ....$
3. 将上述嵌入向量取平均：$\bar(v) = \frac{\sum_{-m}^{m}v_{c+j}}{2m}$
4. 将平均的嵌入向量与输出层权重矩阵相乘，得到输出打分向量$z = W{'}\bar{v}$。相似的单词该得分应该更高；
5. 应用softmax函数将上述打分向量变为概率输出$y=\textrm{softmax}(z)$。如果预测准确的话，那么概率向量$y$会在独热编码为1的地方具备最高的概率值。

word2vec模型就是要在迭代的过程中，通过不断优化$W$和$W'$两个权重矩阵使得我们的语言模型尽可能地接近实际的数据表现。

下面是一个完整的CBOW模型的keras实现。

In [ ]:
import keras.backend as K
from keras.models import Sequential
from keras.utils.vis_utils import model_to_dot, plot_model
from keras.layers import Dense, Embedding, Lambda
from IPython.display import SVG, display

In [ ]:
V = len(vocabulary2)
embedding_dim = 50
window_size = 5

cbow = Sequential()
cbow.add(Embedding(input_dim=V, output_dim=embedding_dim, input_length=window_size*2, name='Embedding'))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embedding_dim,), name='Averaging'))
cbow.add(Dense(V, activation='softmax', name='Output'))
cbow.compile(loss='categorical_crossentropy', optimizer='adadelta')
display(SVG(model_to_dot(cbow, show_shapes=True).create(prog='dot', format='svg')))

** 跳跃型模型 （Skip-Gram） **

Skip-Gram模型与CBOW正好相反，模型是根据中心词来预测上下文。其基本步骤如下：


其架构如下图所示（图源自于："Deep learning for sentence classification"）。图中标识分别为：
1. $x_{ik}$为独热表示的中心词
2. $W_{V \times N}$为输入层的权重矩阵，大小为$V \times N$，其中V是词典大小，而N是设定的词向量维度
3. $h_i$是由$x_{ik}$与权重矩阵$W$相乘的到的隐含层结果，大小为$1 \times N$的向量
4. $W^{'}_{N \times V}$为输出层的权重矩阵。注意，这里的$W^{'}$不是$W$矩阵的转置，而是另外一个全新的矩阵。
5. $y_{Cj}$为待预测的上下文词的独热表示

<img src="./pics/Chapter1-SkipGram.png" width="400">


Initialization
Step 1

    Fix the hyperparameters such as window size (2 * n + 1), the size of feature vectors (N), learning rate, number of epochs etc.
    Initialize other parameters such as the connection weights between the input layer and hidden layer, and hidden layer and the softmax layers (can be random initialization).

Step 2

From each sentence in the input file, form a multi-set which will be a collection of every possible contiguous sub-sequence of size (2 * n + 1). Union of all such multi-sets (corresponding to each sentence) will be our training corpus (which is also a multi-set of windows).
Step 3

Make a vocabulary by collecting all possible distinct words. Let the Vocabulary be V.
Step 4 [redundant step added for clarity in understanding]

To each word in dictionary assign a random feature vector. If x is the word then Vec[x] represents the feature vector. The ith row of weight matrix between the input layer and hidden layer stores the feature vector for ith word in the vocabulary, hence in step 1 we have already initialized the word vectors for each word in the dictionary.
One Training Step

Input: Given a single window  w-n  …  w-1  x  w1  ….  wn  from the training corpus.
Forward Step

 

Input Layer

    Activation Function: Identity
    Input: One hot representation of word x
    Weight Matrix Size: [V * N]
    Objective: Select vector Vec[x] corresponding to x

By feeding one hot representation of word x, the input vector to the hidden layer will be the feature vector corresponding to the word x.

Note: The ith row of the hidden weight matrix is the word vector of the ith word in the vocabulary.

 
Hidden Layer

    Activation Function: Identity
    Input: Word vector corresponding to x
    Weight Matrix Size: [N * V] (for each of 2 * n softmax unit)

There are 2 * n softmax units (each having |V| nodes) in the output layer. Let the softmax units be S-n … S-1 ,S1  …. Sn. Feed Vec[x] to each of the softmax unit (each with its own weight matrix). zth node of softmax unit i tries to predict the probability of zth word in dictionary appearing in context position i with respect to the current context. Let P(i, z) be the output of zth node of the ith softmax unit.

 
Output Layer

The following bullet points are with respect to a single softmax unit (say ith) unit in the output layer:

    Input : Output of hidden layer.
    Objective (Redundant): zth node of softmax unit i tries to predict the probability of zth word in dictionary appearing in context position i with respect to the current context.

Crucial Step: For each context position in set P, add the log of the output of each corresponding context word’s node. So let S = $latex \sum_{i = -n, i \neq 0}^{i = n} \log P(i, wi) $ where  wi is the  ith context word and i ∈ P (Objective here is to maximize this log likelihood of the observation).
Backward Step (Back-Propagation Step):

Objective: Try to maximize S (log likelihood of observation as defined above).

S is a function of weights of the model, all words in vocabulary and the word vector corresponding to x. Using back propagation we tune weights of the model (specifically all weight between hidden layer and output layer) and improve the feature vectors word x (which is also embodied in model in the form of weight between input layer and hidden layer), so as to maximize S. This is same as minimizing the negative log likelihood of the observation.

Redundant Note: During one training step, in a window where x forms the middle word, only the weights between hidden layer and output layer, and feature vector of x will be improved. Feature vectors of all other words will not change (Do some math yourself :p). From the next training instance, the modified feature vector of x will be used as its feature vector. In this way, a feature vector gets incrementally.
Training

Run the training step on the training corpus for several epochs.
Output VS Outcome

Output (of the model): Given a window (a training instance), we get the output as the values which we get at the output nodes. The output of the nodes (of output layer) tries to estimate probabilities. These probabilities can be useful for language modeling, though it is generally not used.

Outcome: After the model is trained completely, the  ith rows of the weight matrix between input layer and hidden layer is the feature vector for the ith word in the dictionary. These feature vectors for each word in dictionary are the outcome of the model. And Word2vec is famous good feature vectors.

## 中文字符预处理：

如前所述在我们编写自己的CBOW或者SkipGram模型之前，我们先要对中文字符进行处理。虽然前面我们已经有了一些方法来处理中文，但是我们想能不能将中文处理纳入keras的文字预处理模块来整体考虑？下面我们将对keras的文字预处理模块进行改造，使其标注类（Tokenizer）能顺利处理中文。我们将从一下几个方面着手。

1. 标注化（tokenizer）
2. 序列化
3. 建立字典
4. 建立正向、反向查阅表

In [22]:
from collections import defaultdict
from collections import OrderedDict
def cntext_to_word_sequence(text, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',):
    translate_dict = dict((c, ' ') for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    words = jieba.cut(text)
    seq = [w for w in words if w]
    return seq

def fit_on_cntexts(texts, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'):
    '''
    假设输入是一组中文列表
    '''
    index_docs = defaultdict(int)
    word_docs = defaultdict(int)
    word_counts = OrderedDict()
    word_index = dict()
    index_word = dict()
    document_count = 0
    for text in texts:
        document_count += 1
        #rint(text)
        
        if isinstance(text, list):
            longtext = ' '.join(text)           
            text = longtext
        seq = cntext_to_word_sequence(text, filters)
        for w in seq:
            if w in word_counts:
                word_counts[w] += 1
            else:
                word_counts[w] = 1
        for w in set(seq):
        # In how many documents each word occurs
           word_docs[w] += 1

        wcounts = list(word_counts.items())
        wcounts.sort(key=lambda x: x[1], reverse=True)
        # forcing the oov_token to index 1 if it exists        
        if oov_token ==None:
            sorted_voc = []
        else:
            sorted_voc = [oov_token]
        sorted_voc.extend(wc[0] for wc in wcounts)
        #rint(sorted_voc)

        # note that index 0 is reserved, never assigned to an existing word
        word_index = dict(
            list(zip(sorted_voc, list(range(1, len(sorted_voc) + 1))))
        )

        index_word = dict((c, w) for w, c in word_index.items())

        for w, c in list(word_docs.items()):
            index_docs[word_index[w]] = c

    return seq, word_index, index_word, word_docs, index_docs, word_counts, document_count
    
def cntexts_to_sequences(texts, num_words, oov_token, document_count, filters, word_index ):
    """Transforms each text in texts to a sequence of integers.
    Only top `num_words-1` most frequent words will be taken into account.
    Only words known by the tokenizer will be taken into account.
    # Arguments
        texts: A list of texts (strings).
    # Returns
        A list of sequences.
    """
    return list(cntexts_to_sequences_generator(texts, num_words, oov_token, document_count, filters, word_index))    


def cntexts_to_sequences_generator(texts, num_words, oov_token, document_count, filters, word_index ):
    """Transforms each Chinese text in `texts` to a sequence of integers.
    Each item in texts can also be a list,
    in which case we assume each item of that list to be a token.
    Only top `num_words-1` most frequent words will be taken into account.
    Only words known by the tokenizer will be taken into account.
    # Arguments
        texts: A list of texts (strings).
    # Yields
        Yields individual sequences.
    """
    num_words = num_words
    oov_token_index = word_index.get(oov_token)
    for text in texts:
        document_count += 1
        #print(text)

        if isinstance(text, list):
            longtext = ' '.join(text)           
            text = longtext
        seq = cntext_to_word_sequence(text, filters)
        #print(seq, '\n')
                
        vect = []
        temp = []
        for w in seq:
            i = word_index.get(w)
            temp.append(i)
            if i is not None:
                if num_words and i >= num_words:
                    if oov_token_index is not None:
                        vect.append(oov_token_index)
                else:
                    vect.append(i)
            elif oov_token is not None:
                vect.append(oov_token_index)
        #print(temp)
        #print('-------------')
        yield vect       

        
class Tokenizer(object):
    """Text tokenization utility class.
    This class allows to vectorize a text corpus, by turning each
    text into either a sequence of integers (each integer being the index
    of a token in a dictionary) or into a vector where the coefficient
    for each token could be binary, based on word count, based on tf-idf...
    # Arguments
        num_words: the maximum number of words to keep, based
            on word frequency. Only the most common `num_words-1` words will
            be kept.
        filters: a string where each element is a character that will be
            filtered from the texts. The default is all punctuation, plus
            tabs and line breaks, minus the `'` character.
        lower: boolean. Whether to convert the texts to lowercase.
        split: str. Separator for word splitting.
        char_level: if True, every character will be treated as a token.
        oov_token: if given, it will be added to word_index and used to
            replace out-of-vocabulary words during text_to_sequence calls
    By default, all punctuation is removed, turning the texts into
    space-separated sequences of words
    (words maybe include the `'` character). These sequences are then
    split into lists of tokens. They will then be indexed or vectorized.
    `0` is a reserved index that won't be assigned to any word.
    """

    def __init__(self, num_words=None,
                 filters='!"#$%&()*+，。；,-./:;<=>?@[\\]^_`{|}~\t\n',
                 lower=True,
                 split=' ',
                 char_level=False,
                 oov_token=None,
                 document_count=0,
                 **kwargs):
        # Legacy support
        if 'nb_words' in kwargs:
            warnings.warn('The `nb_words` argument in `Tokenizer` '
                          'has been renamed `num_words`.')
            num_words = kwargs.pop('nb_words')
        if kwargs:
            raise TypeError('Unrecognized keyword arguments: ' + str(kwargs))

        self.word_counts = OrderedDict()
        self.word_docs = defaultdict(int)
        self.filters = filters
        self.split = split
        self.lower = lower
        self.num_words = num_words
        self.document_count = document_count
        self.char_level = char_level
        self.oov_token = oov_token
        self.index_docs = defaultdict(int)
        self.word_index = dict()
        self.index_word = dict()

    def fit_on_texts(self, texts):
        """Updates internal vocabulary based on a list of texts.
        In the case where texts contains lists,
        we assume each entry of the lists to be a token.
        Required before using `texts_to_sequences` or `texts_to_matrix`.
        # Arguments
            texts: can be a list of strings,
                a generator of strings (for memory-efficiency),
                or a list of list of strings.
        """
        for text in texts:
            self.document_count += 1
            if self.char_level or isinstance(text, list):
                if self.lower:
                    if isinstance(text, list):
                        text = [text_elem.lower() for text_elem in text]
                    else:
                        text = text.lower()
                seq = text
            else:
                seq = text_to_word_sequence(text,
                                            self.filters,
                                            self.lower,
                                            self.split)
            for w in seq:
                if w in self.word_counts:
                    self.word_counts[w] += 1
                else:
                    self.word_counts[w] = 1
            for w in set(seq):
                # In how many documents each word occurs
                self.word_docs[w] += 1

        wcounts = list(self.word_counts.items())
        wcounts.sort(key=lambda x: x[1], reverse=True)
        # forcing the oov_token to index 1 if it exists
        if self.oov_token is None:
            sorted_voc = []
        else:
            sorted_voc = [self.oov_token]
        sorted_voc.extend(wc[0] for wc in wcounts)

        # note that index 0 is reserved, never assigned to an existing word
        self.word_index = dict(
            list(zip(sorted_voc, list(range(1, len(sorted_voc) + 1)))))

        self.index_word = dict((c, w) for w, c in self.word_index.items())

        for w, c in list(self.word_docs.items()):
            self.index_docs[self.word_index[w]] = c
            
    def fit_on_cntexts(self, texts):
        '''
        假设输入是一组中文列表
        '''
        for text in texts:
            self.document_count += 1

            if isinstance(text, list):
                longtext = ' '.join(text)           
                text = longtext
            seq = cntext_to_word_sequence(text, self.filters)
            for w in seq:
                if w in self.word_counts:
                    self.word_counts[w] += 1
                else:
                    self.word_counts[w] = 1
            for w in set(seq):
            # In how many documents each word occurs
               self.word_docs[w] += 1

        wcounts = list(self.word_counts.items())
        wcounts.sort(key=lambda x: x[1], reverse=True)
        # forcing the oov_token to index 1 if it exists        
        if self.oov_token is None:
            sorted_voc = []
        else:
            sorted_voc = [self.oov_token]
        sorted_voc.extend(wc[0] for wc in wcounts)

        # note that index 0 is reserved, never assigned to an existing word
        self.word_index = dict(
                list(zip(sorted_voc, list(range(1, len(sorted_voc) + 1))))
        )

        self.index_word = dict((c, w) for w, c in self.word_index.items())

        for w, c in list(self.word_docs.items()):
            self.index_docs[self.word_index[w]] = c

    def fit_on_sequences(self, sequences):
        """Updates internal vocabulary based on a list of sequences.
        Required before using `sequences_to_matrix`
        (if `fit_on_texts` was never called).
        # Arguments
            sequences: A list of sequence.
                A "sequence" is a list of integer word indices.
        """
        self.document_count += len(sequences)
        for seq in sequences:
            seq = set(seq)
            for i in seq:
                self.index_docs[i] += 1

    def texts_to_sequences(self, texts):
        """Transforms each text in texts to a sequence of integers.
        Only top `num_words-1` most frequent words will be taken into account.
        Only words known by the tokenizer will be taken into account.
        # Arguments
            texts: A list of texts (strings).
        # Returns
            A list of sequences.
        """
        return list(self.texts_to_sequences_generator(texts))
    
    def cntexts_to_sequences(self, texts):
        """Transforms each text in texts to a sequence of integers.
        Only top `num_words-1` most frequent words will be taken into account.
        Only words known by the tokenizer will be taken into account.
        # Arguments
            texts: A list of texts (strings).
        # Returns
            A list of sequences.
        """
        return list(self.cntexts_to_sequences_generator(texts))    

    def texts_to_sequences_generator(self, texts):
        """Transforms each text in `texts` to a sequence of integers.
        Each item in texts can also be a list,
        in which case we assume each item of that list to be a token.
        Only top `num_words-1` most frequent words will be taken into account.
        Only words known by the tokenizer will be taken into account.
        # Arguments
            texts: A list of texts (strings).
        # Yields
            Yields individual sequences.
        """
        num_words = self.num_words
        oov_token_index = self.word_index.get(self.oov_token)
        
        for text in texts:
            if self.char_level or isinstance(text, list):
                if self.lower:
                    if isinstance(text, list):
                        text = [text_elem.lower() for text_elem in text]
                    else:
                        text = text.lower()
                seq = text
            else:
                seq = text_to_word_sequence(text,
                                            self.filters,
                                            self.lower,
                                            self.split)
            vect = []
            for w in seq:
                i = self.word_index.get(w)
                if i is not None:
                    if num_words and i >= num_words:
                        if oov_token_index is not None:
                            vect.append(oov_token_index)
                    else:
                        vect.append(i)
                elif self.oov_token is not None:
                    vect.append(oov_token_index)
            yield vect
            
    def cntexts_to_sequences_generator(self, texts):
        """Transforms each text in `texts` to a sequence of integers.
        Each item in texts can also be a list,
        in which case we assume each item of that list to be a token.
        Only top `num_words-1` most frequent words will be taken into account.
        Only words known by the tokenizer will be taken into account.
        # Arguments
            texts: A list of texts (strings).
        # Yields
            Yields individual sequences.
        """
        num_words = self.num_words
        oov_token_index = self.word_index.get(self.oov_token)
        for text in texts:
            self.document_count += 1

            if isinstance(text, list):
                longtext = ' '.join(text)           
                text = longtext
            seq = cntext_to_word_sequence(text, self.filters)
                    
            vect = []
            for w in seq:
                i = self.word_index.get(w)
                if i is not None:
                    if num_words and i >= num_words:
                        if oov_token_index is not None:
                            vect.append(oov_token_index)
                    else:
                        vect.append(i)
                elif self.oov_token is not None:
                    vect.append(oov_token_index)
            yield vect            

    def sequences_to_texts(self, sequences):
        """Transforms each sequence into a list of text.
        Only top `num_words-1` most frequent words will be taken into account.
        Only words known by the tokenizer will be taken into account.
        # Arguments
            texts: A list of sequences (list of integers).
        # Returns
            A list of texts (strings)
        """
        return list(self.sequences_to_texts_generator(sequences))

    def sequences_to_texts_generator(self, sequences):
        """Transforms each sequence in `sequences` to a list of texts(strings).
        Each sequence has to a list of integers.
        In other words, sequences should be a list of sequences
        Only top `num_words-1` most frequent words will be taken into account.
        Only words known by the tokenizer will be taken into account.
        # Arguments
            texts: A list of sequences.
        # Yields
            Yields individual texts.
        """
        num_words = self.num_words
        oov_token_index = self.word_index.get(self.oov_token)
        for seq in sequences:
            vect = []
            for num in seq:
                word = self.index_word.get(num)
                if word is not None:
                    if num_words and num >= num_words:
                        if oov_token_index is not None:
                            vect.append(self.index_word[oov_token_index])
                    else:
                        vect.append(word)
                elif self.oov_token is not None:
                    vect.append(self.index_word[oov_token_index])
            vect = ' '.join(vect)
            yield vect

    def texts_to_matrix(self, texts, mode='binary'):
        """Convert a list of texts to a Numpy matrix.
        # Arguments
            texts: list of strings.
            mode: one of "binary", "count", "tfidf", "freq".
        # Returns
            A Numpy matrix.
        """
        sequences = self.texts_to_sequences(texts)
        return self.sequences_to_matrix(sequences, mode=mode)

    def sequences_to_matrix(self, sequences, mode='binary'):
        """Converts a list of sequences into a Numpy matrix.
        # Arguments
            sequences: list of sequences
                (a sequence is a list of integer word indices).
            mode: one of "binary", "count", "tfidf", "freq"
        # Returns
            A Numpy matrix.
        # Raises
            ValueError: In case of invalid `mode` argument,
                or if the Tokenizer requires to be fit to sample data.
        """
        if not self.num_words:
            if self.word_index:
                num_words = len(self.word_index) + 1
            else:
                raise ValueError('Specify a dimension (`num_words` argument), '
                                 'or fit on some text data first.')
        else:
            num_words = self.num_words

        if mode == 'tfidf' and not self.document_count:
            raise ValueError('Fit the Tokenizer on some data '
                             'before using tfidf mode.')

        x = np.zeros((len(sequences), num_words))
        for i, seq in enumerate(sequences):
            if not seq:
                continue
            counts = defaultdict(int)
            for j in seq:
                if j >= num_words:
                    continue
                counts[j] += 1
            for j, c in list(counts.items()):
                if mode == 'count':
                    x[i][j] = c
                elif mode == 'freq':
                    x[i][j] = c / len(seq)
                elif mode == 'binary':
                    x[i][j] = 1
                elif mode == 'tfidf':
                    # Use weighting scheme 2 in
                    # https://en.wikipedia.org/wiki/Tf%E2%80%93idf
                    tf = 1 + np.log(c)
                    idf = np.log(1 + self.document_count /
                                 (1 + self.index_docs.get(j, 0)))
                    x[i][j] = tf * idf
                else:
                    raise ValueError('Unknown vectorization mode:', mode)
        return x

    def get_config(self):
        '''Returns the tokenizer configuration as Python dictionary.
        The word count dictionaries used by the tokenizer get serialized
        into plain JSON, so that the configuration can be read by other
        projects.
        # Returns
            A Python dictionary with the tokenizer configuration.
        '''
        json_word_counts = json.dumps(self.word_counts)
        json_word_docs = json.dumps(self.word_docs)
        json_index_docs = json.dumps(self.index_docs)
        json_word_index = json.dumps(self.word_index)
        json_index_word = json.dumps(self.index_word)

        return {
            'num_words': self.num_words,
            'filters': self.filters,
            'lower': self.lower,
            'split': self.split,
            'char_level': self.char_level,
            'oov_token': self.oov_token,
            'document_count': self.document_count,
            'word_counts': json_word_counts,
            'word_docs': json_word_docs,
            'index_docs': json_index_docs,
            'index_word': json_index_word,
            'word_index': json_word_index
        }

    def to_json(self, **kwargs):
        """Returns a JSON string containing the tokenizer configuration.
        To load a tokenizer from a JSON string, use
        `keras.preprocessing.text.tokenizer_from_json(json_string)`.
        # Arguments
            **kwargs: Additional keyword arguments
                to be passed to `json.dumps()`.
        # Returns
            A JSON string containing the tokenizer configuration.
        """
        config = self.get_config()
        tokenizer_config = {
            'class_name': self.__class__.__name__,
            'config': config
        }
        return json.dumps(tokenizer_config, **kwargs)


def tokenizer_from_json(json_string):
    """Parses a JSON tokenizer configuration file and returns a
    tokenizer instance.
    # Arguments
        json_string: JSON string encoding a tokenizer configuration.
    # Returns
        A Keras Tokenizer instance
    """
    tokenizer_config = json.loads(json_string)
    config = tokenizer_config.get('config')

    word_counts = json.loads(config.pop('word_counts'))
    word_docs = json.loads(config.pop('word_docs'))
    index_docs = json.loads(config.pop('index_docs'))
    # Integer indexing gets converted to strings with json.dumps()
    index_docs = {int(k): v for k, v in index_docs.items()}
    index_word = json.loads(config.pop('index_word'))
    index_word = {int(k): v for k, v in index_word.items()}
    word_index = json.loads(config.pop('word_index'))

    tokenizer = Tokenizer(**config)
    tokenizer.word_counts = word_counts
    tokenizer.word_docs = word_docs
    tokenizer.index_docs = index_docs
    tokenizer.word_index = word_index
    tokenizer.index_word = index_word

    return tokenizer

In [26]:
from collections import defaultdict
from collections import OrderedDict
def cntext_to_word_sequence(text, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',):
    translate_dict = dict((c, ' ') for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    words = jieba.cut(text)
    seq = [w for w in words if w]
    return seq

    
def cntexts_to_sequences(texts, num_words, oov_token, document_count, filters, word_index ):
    """Transforms each text in texts to a sequence of integers.
    Only top `num_words-1` most frequent words will be taken into account.
    Only words known by the tokenizer will be taken into account.
    # Arguments
        texts: A list of texts (strings).
    # Returns
        A list of sequences.
    """
    return list(cntexts_to_sequences_generator(texts, num_words, oov_token, document_count, filters, word_index))    


def cntexts_to_sequences_generator(texts, num_words, oov_token, document_count, filters, word_index ):
    """Transforms each Chinese text in `texts` to a sequence of integers.
    Each item in texts can also be a list,
    in which case we assume each item of that list to be a token.
    Only top `num_words-1` most frequent words will be taken into account.
    Only words known by the tokenizer will be taken into account.
    # Arguments
        texts: A list of texts (strings).
    # Yields
        Yields individual sequences.
    """
    num_words = num_words
    oov_token_index = word_index.get(oov_token)
    for text in texts:
        document_count += 1
        #print(text)

        if isinstance(text, list):
            longtext = ' '.join(text)           
            text = longtext
        seq = cntext_to_word_sequence(text, filters)
        #print(seq, '\n')
                
        vect = []
        temp = []
        for w in seq:
            i = word_index.get(w)
            temp.append(i)
            if i is not None:
                if num_words and i >= num_words:
                    if oov_token_index is not None:
                        vect.append(oov_token_index)
                else:
                    vect.append(i)
            elif oov_token is not None:
                vect.append(oov_token_index)
        #print(temp)
        #print('-------------')
        yield vect       

        
class cnTokenizer(Tokenizer):
    """Text tokenization utility class.
    This class allows to vectorize a text corpus, by turning each
    text into either a sequence of integers (each integer being the index
    of a token in a dictionary) or into a vector where the coefficient
    for each token could be binary, based on word count, based on tf-idf...
    # Arguments
        num_words: the maximum number of words to keep, based
            on word frequency. Only the most common `num_words-1` words will
            be kept.
        filters: a string where each element is a character that will be
            filtered from the texts. The default is all punctuation, plus
            tabs and line breaks, minus the `'` character.
        lower: boolean. Whether to convert the texts to lowercase.
        split: str. Separator for word splitting.
        char_level: if True, every character will be treated as a token.
        oov_token: if given, it will be added to word_index and used to
            replace out-of-vocabulary words during text_to_sequence calls
    By default, all punctuation is removed, turning the texts into
    space-separated sequences of words
    (words maybe include the `'` character). These sequences are then
    split into lists of tokens. They will then be indexed or vectorized.
    `0` is a reserved index that won't be assigned to any word.
    """

    def __init__(self, num_words=None,
                 filters='!"#$%&()*+，。；,-./:;<=>?@[\\]^_`{|}~\t\n',
                 lower=True,
                 split=' ',
                 char_level=False,
                 oov_token=None,
                 document_count=0,
                 **kwargs):
        # Legacy support
        if 'nb_words' in kwargs:
            warnings.warn('The `nb_words` argument in `Tokenizer` '
                          'has been renamed `num_words`.')
            num_words = kwargs.pop('nb_words')
        if kwargs:
            raise TypeError('Unrecognized keyword arguments: ' + str(kwargs))

        self.word_counts = OrderedDict()
        self.word_docs = defaultdict(int)
        self.filters = filters
        self.split = split
        self.lower = lower
        self.num_words = num_words
        self.document_count = document_count
        self.char_level = char_level
        self.oov_token = oov_token
        self.index_docs = defaultdict(int)
        self.word_index = dict()
        self.index_word = dict()
            
    def fit_on_cntexts(self, texts):
        '''
        假设输入是一组中文列表
        '''
        for text in texts:
            self.document_count += 1

            if isinstance(text, list):
                longtext = ' '.join(text)           
                text = longtext
            seq = cntext_to_word_sequence(text, self.filters)
            for w in seq:
                if w in self.word_counts:
                    self.word_counts[w] += 1
                else:
                    self.word_counts[w] = 1
            for w in set(seq):
            # In how many documents each word occurs
               self.word_docs[w] += 1

        wcounts = list(self.word_counts.items())
        wcounts.sort(key=lambda x: x[1], reverse=True)
        # forcing the oov_token to index 1 if it exists        
        if self.oov_token is None:
            sorted_voc = []
        else:
            sorted_voc = [self.oov_token]
        sorted_voc.extend(wc[0] for wc in wcounts)

        # note that index 0 is reserved, never assigned to an existing word
        self.word_index = dict(
                list(zip(sorted_voc, list(range(1, len(sorted_voc) + 1))))
        )

        self.index_word = dict((c, w) for w, c in self.word_index.items())

        for w, c in list(self.word_docs.items()):
            self.index_docs[self.word_index[w]] = c

    
    def cntexts_to_sequences(self, texts):
        """Transforms each text in texts to a sequence of integers.
        Only top `num_words-1` most frequent words will be taken into account.
        Only words known by the tokenizer will be taken into account.
        # Arguments
            texts: A list of texts (strings).
        # Returns
            A list of sequences.
        """
        return list(self.cntexts_to_sequences_generator(texts))    

    def cntexts_to_sequences_generator(self, texts):
        """Transforms each text in `texts` to a sequence of integers.
        Each item in texts can also be a list,
        in which case we assume each item of that list to be a token.
        Only top `num_words-1` most frequent words will be taken into account.
        Only words known by the tokenizer will be taken into account.
        # Arguments
            texts: A list of texts (strings).
        # Yields
            Yields individual sequences.
        """
        num_words = self.num_words
        oov_token_index = self.word_index.get(self.oov_token)
        for text in texts:
            self.document_count += 1

            if isinstance(text, list):
                longtext = ' '.join(text)           
                text = longtext
            seq = cntext_to_word_sequence(text, self.filters)
                    
            vect = []
            for w in seq:
                i = self.word_index.get(w)
                if i is not None:
                    if num_words and i >= num_words:
                        if oov_token_index is not None:
                            vect.append(oov_token_index)
                    else:
                        vect.append(i)
                elif self.oov_token is not None:
                    vect.append(oov_token_index)
            yield vect            




下面我们实验一下

In [30]:
documents = [u'独热编码是对于当前单词表中的单词使用1个向量进行表达的简单方式，独热编码有自身的缺点和有点。', 
             u'研究者探索了可以解决存储效率和单词之间关系问题的方法', 
             u'独热编码虽然简单', 
             u'假如当前的单词表有10个不同的单词，并且每个单词都不一样']

corpus = documents
tokenizer = cnTokenizer()
tokenizer.fit_on_cntexts(documents)
V = len(tokenizer.word_index) + 1
text_sequences=tokenizer.cntexts_to_sequences(documents)
#seq, word_index, index_word, word_docs, index_docs, word_counts, document_count = fit_on_cntexts(corpus)
print(tokenizer.index_docs,'\n')
print('---------------')
print(tokenizer.word_index, '\n')
print(tokenizer.index_word)
print(tokenizer.word_docs)
print(tokenizer.index_docs)
print(tokenizer.word_counts)
print('\n')
print(text_sequences)

defaultdict(<class 'int'>, {1: 3, 20: 1, 11: 2, 9: 2, 5: 2, 4: 2, 8: 2, 10: 2, 3: 2, 18: 1, 23: 1, 2: 3, 14: 1, 17: 1, 13: 1, 21: 1, 22: 1, 16: 1, 7: 2, 12: 1, 19: 1, 15: 1, 6: 2, 28: 1, 33: 1, 34: 1, 31: 1, 32: 1, 27: 1, 30: 1, 29: 1, 25: 1, 24: 1, 26: 1, 35: 1, 37: 1, 36: 1, 40: 1, 41: 1, 38: 1, 43: 1, 39: 1, 42: 1}) 

---------------
{'的': 1, '单词': 2, '独热': 3, '编码': 4, ' ': 5, '当前': 6, '单词表': 7, '个': 8, '简单': 9, '有': 10, '和': 11, '是': 12, '对于': 13, '中': 14, '使用': 15, '1': 16, '向量': 17, '进行': 18, '表达': 19, '方式': 20, '自身': 21, '缺点': 22, '有点': 23, '研究者': 24, '探索': 25, '了': 26, '可以': 27, '解决': 28, '存储': 29, '效率': 30, '之间': 31, '关系': 32, '问题': 33, '方法': 34, '虽然': 35, '假如': 36, '10': 37, '不同': 38, '并且': 39, '每个': 40, '都': 41, '不': 42, '一样': 43} 

{1: '的', 2: '单词', 3: '独热', 4: '编码', 5: ' ', 6: '当前', 7: '单词表', 8: '个', 9: '简单', 10: '有', 11: '和', 12: '是', 13: '对于', 14: '中', 15: '使用', 16: '1', 17: '向量', 18: '进行', 19: '表达', 20: '方式', 21: '自身', 22: '缺点', 23: '有点', 24: '研究者', 25: '探索', 26: '了', 2

现在我们已经对keras的标准标注类做了必要的小修改，使得其能顺利处理中文文档。

## CBOW的keras实现

下面我们在keras中来实现基于CBOW算法的word2vec模型。我们需要进行如下的设计：
1. 首先我们要将原始输入的文字进行标注化（tokenize），下标代表每一个单词，构造单词表；
2. 构造一个CBOW数据生成器，能产出（上下文，目标单词）的组合
3. 构造CBOW的网络结构
4. 训练CBOW模型
5. 获取词向量

In [ ]:
# 步骤 1
from keras.preprocessing import text
from keras.utils import np_utils
from keras.preprocessing import sequence

tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(norm_bible)
word2id = tokenizer.word_index

# build vocabulary of unique words
word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in norm_bible]

vocab_size = len(word2id)
embed_size = 100
window_size = 2 # context window size

print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

In [ ]:
# 步骤 2
def generate_context_word_pairs(corpus, window_size, vocab_size):
    context_length = window_size*2
    for words in corpus:
        sentence_length = len(words)
        for index, word in enumerate(words):
            context_words = []
            label_word   = []            
            start = index - window_size
            end = index + window_size + 1
            
            context_words.append([words[i] 
                                 for i in range(start, end) 
                                 if 0 <= i < sentence_length 
                                 and i != index])
            label_word.append(word)

            x = sequence.pad_sequences(context_words, maxlen=context_length)
            y = np_utils.to_categorical(label_word, vocab_size)
            yield (x, y)
            
            
# Test this out for some samples
i = 0
for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
    if 0 not in x[0]:
        print('Context (X):', [id2word[w] for w in x[0]], '-> Target (Y):', id2word[np.argwhere(y[0])[0][0]])
    
        if i == 10:
            break
i += 1

In [32]:
# 步骤 3
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

# build CBOW architecture
cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# view model summary
print(cbow.summary())

# visualize model structure
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(cbow, show_shapes=True, show_layer_names=False, 
rankdir='TB').create(prog='dot', format='svg'))

NameError: name 'vocab_size' is not defined

<img src="./pics/Chapter1-CBOW-model.png" width="400">

In [ ]:
# 步骤 4
for epoch in range(1, 6):
    loss = 0.
    i = 0
    for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
        i += 1
        loss += cbow.train_on_batch(x, y)
        if i % 100000 == 0:
            print('Processed {} (context, word) pairs'.format(i))

    print('Epoch:', epoch, '\tLoss:', loss)
print()

In [34]:
# 步骤 5
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

pd.DataFrame(weights, index=list(id2word.values())[1:]).head()

IndexError: list index out of range

In [128]:
from __future__ import absolute_import

from keras import backend as K
import numpy as np
from keras.utils.np_utils import accuracy
from keras.models import Sequential, Model
from keras.layers import Input, Lambda, Dense, merge
from keras.layers.embeddings import Embedding
from keras.optimizers import SGD
from keras.objectives import mse

import global_settings as G
from sentences_generator import Sentences
import vocab_generator as V_gen
import save_embeddings as S

k = G.window_size # context windows size
context_size = 2*k

# Creating a sentence generator from demo file
sentences = Sentences("test_file.txt")
vocabulary = dict()
V_gen.build_vocabulary(vocabulary, sentences)
V_gen.filter_vocabulary_based_on(vocabulary, G.min_count)
reverse_vocabulary = V_gen.generate_inverse_vocabulary_lookup(vocabulary, "vocab.txt")

# generate embedding matrix with all values between -1/2d, 1/2d
embedding = np.random.uniform(-1.0/2.0/G.embedding_dimension, 1.0/2.0/G.embedding_dimension, (G.vocab_size+3, G.embedding_dimension))

# Creating CBOW model
# Model has 3 inputs
# Current word index, context words indexes and negative sampled word indexes
word_index = Input(shape=(1,))
context = Input(shape=(context_size,))
negative_samples = Input(shape=(G.negative,))
# All the inputs are processed through a common embedding layer
shared_embedding_layer = Embedding(input_dim=(G.vocab_size+3), output_dim=G.embedding_dimension, weights=[embedding])
word_embedding = shared_embedding_layer(word_index)
context_embeddings = shared_embedding_layer(context)
negative_words_embedding = shared_embedding_layer(negative_samples)
# Now the context words are averaged to get the CBOW vector
cbow = Lambda(lambda x: K.mean(x, axis=1), output_shape=(G.embedding_dimension,))(context_embeddings)
# The context is multiplied (dot product) with current word and negative sampled words
word_context_product = merge([word_embedding, cbow], mode='dot')
negative_context_product = merge([negative_words_embedding, cbow], mode='dot', concat_axis=-1)
# The dot products are outputted
model = Model(input=[word_index, context, negative_samples], output=[word_context_product, negative_context_product])
# binary crossentropy is applied on the output
model.compile(optimizer='rmsprop', loss='binary_crossentropy')
print(model.summary())

# model.fit_generator(V_gen.pretraining_batch_generator(sentences, vocabulary, reverse_vocabulary), samples_per_epoch=G.train_words, nb_epoch=1)
model.fit_generator(V_gen.pretraining_batch_generator(sentences, vocabulary, reverse_vocabulary), samples_per_epoch=10, nb_epoch=1)
# Save the trained embedding
S.save_embeddings("embedding.txt", shared_embedding_layer.get_weights()[0], vocabulary)


D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


ImportError: cannot import name 'accuracy'

## Skip-Gram模型的keras实现


In [ ]:
import numpy as np
np.random.seed(13)
from keras.models import Sequential
from keras.layers import Embedding, Merge, Reshape, Activation, Flatten, Input, merge, Dense
from keras.layers.core import Lambda
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.utils.visualize_util import model_to_dot, plot
from keras.preprocessing.text import Tokenizer, base_filter
from keras.preprocessing.sequence import skipgrams, pad_sequences
from keras import backend as K
from keras.models import Model

from gensim.models.doc2vec import Word2Vec
import pandas as pd

In [ ]:
corpus = [sentence for sentence in data2['text'] if sentence.count(" ") >= 2]
tokenizer = Tokenizer(filters=base_filter()+"'")
tokenizer.fit_on_texts(corpus)
V = len(tokenizer.word_index) + 1
vocab = tokenizer.word_index.keys()

In [ ]:
maxlen = 10
dim_embeddings = 128
inputs = Input(shape=(maxlen,), name = 'inputWord',dtype='int32')
context = Input(shape=(maxlen,), name = 'contextWord',dtype='int32')

embedded_sequences_input = Embedding(200000,
                                     dim_embeddings,
                                     input_length=maxlen,
                                    name='input_embeddings',
                                    trainable=True)(inputs)
embedded_sequences_context = Embedding(200000,
                                       dim_embeddings,
                                       input_length=maxlen,
                                       trainable=True,
                                       name='context_embeddings')(context)

embedded_sequences_context1 = Lambda(lambda s: K.sum(s, axis=1), output_shape=lambda s: (s[0],s[2]))(embedded_sequences_context)
embedded_sequences_input1 = Lambda(lambda s: K.sum(s, axis=1), output_shape=lambda s: (s[0],s[2]))(embedded_sequences_input)

# embedded_sequences_input1 = Reshape((1,), input_shape=(1,128))(embedded_sequences_input1)
# embedded_sequences_context1 = Reshape((1,), input_shape=(1,128))(embedded_sequences_context1)

final = merge([embedded_sequences_input1, embedded_sequences_context1], mode='dot', dot_axes=1)
# final = Reshape((1,), input_shape=(1,1))(final)
final = Dense(1, activation='sigmoid')(final)
model = Model([inputs, context], final)
model.compile(loss="binary_crossentropy", optimizer="rmsprop")
display(SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg')))

In [ ]:
corpus = [sentence for sentence in data1['text'] if sentence.count(" ") >= 2]
tokenizer = Tokenizer(filters=base_filter()+"'")
tokenizer.fit_on_texts(corpus)
V = len(tokenizer.word_index) + 1
maxlen = 10
for _ in range(1):
    loss = 0.
    for doc in tokenizer.texts_to_sequences(data1['text']):
#         print doc
        data, labels = skipgrams(sequence=doc, vocabulary_size=V, window_size=5, negative_samples=5.)
        ngram_representation = []
        ngram_contexts = []
        ngram_targets = []
        for j in data:
            ngram_context_pairs = []
            word1 = new_dict[j[0]]
            word2 = new_dict[j[1]]
            ngram_contexts.append(word1)
            ngram_targets.append(word2)
            ngram_context_pairs.append(word1)
            ngram_context_pairs.append(word2)
            ngram_representation.append(ngram_context_pairs)

        ngram_contexts = pad_sequences(ngram_contexts, maxlen=10, dtype='int32')
        ngram_targets = pad_sequences(ngram_targets,maxlen=10,dtype='int32')
        X = [ngram_contexts,ngram_targets]
#         print len(ngram_contexts[0])
        Y = np.array(labels, dtype=np.int32)
#         print ngram_contexts.shape, ngram_targets.shape, Y.shape
        if ngram_contexts.shape[0]!=0:
#             loss += model.train_on_batch(X,Y)
            try:
#                 print "tried"
                loss += model.train_on_batch(X,Y)
            except IndexError:
                continue
    print loss



对计算出来的词向量进行可视化

In [ ]:
def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
  assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
  plt.figure(figsize=(18, 18))  # in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')

  plt.savefig(filename)

try:
  # pylint: disable=g-import-not-at-top
  from sklearn.manifold import TSNE
  import matplotlib.pyplot as plt

  tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
  plot_only = 500
  low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
  labels = [reverse_dictionary[i] for i in xrange(plot_only)]
  plot_with_labels(low_dim_embs, labels)

except ImportError:
print('Please install sklearn, matplotlib, and scipy to show embeddings.')